In [26]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Traditional blur metrics
def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def tenengrad(image):
    gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    grad = np.sqrt(gx**2 + gy**2)
    return np.mean(grad)

def fft_blur(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    return np.mean(magnitude_spectrum)

# Load model
model = load_model('blur_detection_model_v2.h5',compile=False)
model.summary()

# Input image path
image_path = r"E:\DL HACKATHON PROJECT\dataset\sharp\100_NIKON-D3400-35MM_S.JPG"
image = cv2.imread(image_path)

# Check if image was loaded
if image is None:
    raise ValueError("Image could not be loaded. Check path.")

# Resize and preprocess for MobileNetV2
image_resized = cv2.resize(image, (224, 224))
image_array = img_to_array(image_resized)
image_array = preprocess_input(image_array)
image_array = np.expand_dims(image_array, axis=0)

# Predict blur severity (value between 0 and 100)
predicted_blur = model.predict(image_array)[0][0]*5

# Traditional metrics
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
lap_var = variance_of_laplacian(gray)
tenen = tenengrad(gray)
fft_metric = fft_blur(gray)

# Print results
print(f"\nPredicted Blur Severity (0 to 100): {predicted_blur:.2f}")
print(f"Laplacian Variance: {lap_var:.2f}")
print(f"Tenengrad: {tenen:.2f}")
print(f"FFT Blur Metric: {fft_metric:.2f}")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,430,273 (9.27 MB)

 Trainable params: 172,289 (673.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step

Predicted Blur Severity (0 to 100): 17.99
Laplacian Variance: 94.21
Tenengrad: 12.12
FFT Blur Metric: 164.24
